In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
import re
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from concurrent.futures import ThreadPoolExecutor
from shapely.geometry import LineString, Polygon

from tqdm.notebook import tqdm

C:\Users\fhacesga\Anaconda3\envs\deeplearning\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
base_dir = r"C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\"

known_dir = f"{base_dir}Known"
unknown_dir = f"{base_dir}Unknown"
computed_dir = f"{base_dir}Computed"
encoded_dir  = f"{base_dir}Encoded"
bounds_dir = f"{base_dir}BoundTraining"

In [3]:
def preprocess(image):
    image = np.asarray(image) * 255
    image = image.astype(np.uint8)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    _, image = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY_INV)
    image = cv2.Canny(image, 100, 250).astype(np.uint8)
    image = cv2.dilate(image, np.ones((10, 10), np.uint8))
    return image

def save(image, name):
    Image.fromarray(image).save(name)
    
    
def getCenter(image):
    height, width = image.shape[:2]
    y_center = height // 2
    x_center = width // 2
    return y_center, x_center

def bomb_center(image, perc, replacement_value=0, ):
    height, width = image.shape[:2]
    
    y_center, x_center = getCenter(image)
    
    window_size_x = np.int32(width * perc)
    window_size_y = np.int32(height * perc)

    # Calculate the boundaries of the center window
    y_start = y_center - window_size_y // 2
    y_end = y_start + window_size_y
    x_start = x_center - window_size_x // 2
    x_end = x_start + window_size_x

    # Create a copy of the image array to avoid modifying the original array
    modified_image = np.copy(image)

    # Replace the values in the center window with the replacement value
    modified_image[y_start:y_end, x_start:x_end] = replacement_value

    return modified_image

def process_floodfill(image):
    # GET LINES    
    # https://stackoverflow.com/questions/45322630/how-to-detect-lines-in-opencv
    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 5  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = image.shape[0] // 4 # minimum number of pixels making up a line
    max_line_gap = 100 # maximum gap in pixels between connectable line segments
    line_image = np.copy(image) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)
    
    
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
    
    # BOMB CENTER
    bombed = bomb_center(line_image, 0.5, replacement_value=0)
    y_center, x_center = getCenter(bombed)
    
    # GET FILLED
    _, _, filled, _ = cv2.floodFill(bombed, None, (x_center, y_center), 1)
    
    return filled, line_image

def get_bounding_box(image):
    # Find the contours of the True regions
    contours, _ = cv2.findContours(image.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize the minimum and maximum coordinates
    min_x = min_y = float('inf')
    max_x = max_y = 0

    # Iterate over each contour and update the minimum and maximum coordinates
    for contour in contours:
        x, y, width, height = cv2.boundingRect(contour)
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x + width)
        max_y = max(max_y, y + height)

    # Return the bounding box coordinates as a tuple (x, y, width, height)
    return min_x, min_y, max_x - min_x, max_y - min_y

def image_from_bbox(bbox, image_shape):
    # Create an empty boolean image
    image = np.zeros(image_shape, dtype=bool)

    # Extract the bounding box coordinates
    x, y, width, height = bbox

    # Set the region inside the bounding box to True
    image[y:y+height, x:x+width] = True

    return image

def replace_border_pixels(image, n, replacement_value):
    replaced_image = np.copy(image)

    # Replace top and bottom rows
    replaced_image[:n, :] = replacement_value
    replaced_image[-n:, :] = replacement_value

    # Replace left and right columns
    replaced_image[:, :n] = replacement_value
    replaced_image[:, -n:] = replacement_value

    return replaced_image

In [4]:
# Get a list of image files in the folder
image_list = [f for f in os.listdir(bounds_dir) if f.endswith((".png", ".jpg", ".jpeg", ".tif"))]
        
def PrepareDataset(image_file, folder_path=bounds_dir):
    testimg = Image.open(folder_path + "\\" + image_file)

    testimage_prep = preprocess(testimg)

    testimage_out, lines = process_floodfill(testimage_prep)
    
    testimage_out = replace_border_pixels(testimage_out, 4, 0)
    
    bbox = get_bounding_box(testimage_out)
    bbox_image = image_from_bbox(bbox, testimage_out.shape)
    print(bbox)
    
    basename = f"{folder_path}\\{image_file[:-4]}"
    print(basename)
    np.save(f"{basename}_roi.npy", bbox)
    Image.fromarray(bbox_image).save(f"{basename}_roi.png")

In [7]:
folder_path = bounds_dir
# Iterate over files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.tif')):  # Add more image extensions if needed
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        if image is not None:
            height, width = image.shape[:2]
            print(f"Image: {filename}, Size: {width}x{height} pixels")
        else:
            print(f"Unable to read image: {filename}")

Image: 48039C0010K.tif, Size: 14400x10350 pixels
Image: 48039C0015K.tif, Size: 14400x10350 pixels
Image: 48039C0020K.tif, Size: 14400x10350 pixels
Image: 48039C0030K.tif, Size: 14400x10350 pixels
Image: 48039C0035K.tif, Size: 14400x10350 pixels
Image: 48039C0040K.tif, Size: 14400x10350 pixels
Image: 48039C0041K.tif, Size: 14400x10350 pixels
Image: 48039C0042K.tif, Size: 14400x10350 pixels
Image: 48039C0043K.tif, Size: 14400x10350 pixels
Image: 48039C0044K.tif, Size: 14400x10350 pixels
Image: 48039C0065K.tif, Size: 14400x10350 pixels
Image: 48039C0105K.tif, Size: 14400x10350 pixels
Image: 48039C0110K.tif, Size: 14400x10350 pixels
Image: 48071C0160F.png, Size: 9600x14400 pixels
Image: 48071C0170E.png, Size: 14400x10350 pixels
Image: 48071C0180F.png, Size: 9600x14400 pixels
Image: 48071C0190E.png, Size: 14400x10350 pixels
Image: 48071C0335E.png, Size: 14400x10350 pixels
Image: 48071C0345E.png, Size: 14400x10350 pixels
Image: 48071C0355E.png, Size: 14400x10350 pixels
Image: 48071C0365E.png

Image: 48201C0765M.tif, Size: 14400x10350 pixels
Image: 48201C0770N.tif, Size: 14400x10350 pixels
Image: 48201C0785L.tif, Size: 14183x10267 pixels
Image: 48201C0805L.tif, Size: 14200x10250 pixels
Image: 48201C0810L.tif, Size: 14192x10292 pixels
Image: 48201C0830L.tif, Size: 14191x10233 pixels
Image: 48201C0835L.tif, Size: 14184x10258 pixels
Image: 48201C0840L.tif, Size: 14200x10217 pixels
Image: 48201C0845M.png, Size: 9600x14400 pixels
Image: 48201C0855L.tif, Size: 14216x10266 pixels
Image: 48201C0860L.tif, Size: 14192x10267 pixels
Image: 48201C0865M.png, Size: 9600x14400 pixels
Image: 48201C0870M.png, Size: 9600x14400 pixels
Image: 48201C0880M.tif, Size: 14400x10350 pixels
Image: 48201C0885N.png, Size: 9600x14400 pixels
Image: 48201C0890M.png, Size: 9600x14400 pixels
Image: 48201C0895N.png, Size: 9600x14400 pixels
Image: 48201C0905N.png, Size: 9600x14400 pixels
Image: 48201C0910M.tif, Size: 14400x10350 pixels
Image: 48201C0915N.png, Size: 9600x14400 pixels
Image: 48201C0920M.tif, Size

In [5]:
with ThreadPoolExecutor(max_workers=10) as executor:
    for i in tqdm(executor.map(PrepareDataset, image_list), total=len(image_list)):
        i

C:\Users\fhacesga\Anaconda3\envs\deeplearning\lib\site-packages\PIL\Image.py:3077: DecompressionBombWarning: Image size (149040000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


  0%|          | 0/217 [00:00<?, ?it/s]

(3144, 2250, 8117, 5981)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0015K
(2416, 175, 9570, 10008)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0010K
(3137, 2075, 8009, 7552)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0035K
(2445, 176, 9541, 10034)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0030K
(3090, 515, 8228, 9205)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0041K


C:\Users\fhacesga\Anaconda3\envs\deeplearning\lib\site-packages\PIL\Image.py:3077: DecompressionBombWarning: Image size (138240000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


(3147, 2492, 8129, 5980)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0020K
(3079, 504, 8243, 9150)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0043K
(3113, 503, 8203, 9344)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0042K
(3079, 535, 8239, 9318)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0044K
(3090, 712, 8139, 9139)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0040K


C:\Users\fhacesga\Anaconda3\envs\deeplearning\lib\site-packages\PIL\Image.py:3077: DecompressionBombWarning: Image size (149238064 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


(3104, 1163, 8162, 7982)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0105K
(3082, 586, 7829, 9266)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0065K
(3121, 1405, 8192, 8446)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48039C0110K
(3454, 1455, 7838, 7979)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48157C0045L
(2423, 247, 9564, 9866)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48157C0105L
(2423, 247, 9564, 9866)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48157C0135L
(3380, 1239, 7885, 7979)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48157C0040L
(2423, 247, 9564, 9866)
C:\Users\fhacesga\Desktop\FIRMsDigitizing\RECTDNN\data\000_WorkingFiles\\BoundTraining\48157C0110L
(2423, 247, 

TypeError: 'NoneType' object is not iterable